<a href="https://colab.research.google.com/github/AngelitaPrettyciaHarefa/AngelitaPrettyciaHarefa.github.io/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCRAPING DATA APK ASM ONLINE

In [ ]:
!pip install -qq google-play-scraper                                            #Instal google-play-scraper    

In [ ]:
import pandas as pd                                                             #Import Library
import json

from tqdm import tqdm
from google_play_scraper import Sort, reviews, app
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

In [ ]:
from google.colab import drive                                                  #Untuk Akses GDrive
drive.mount('/content/gdrive', force_remount=True)
path = "gdrive/MyDrive/CODING SKRIPSI/"

In [ ]:
app_packages = [                                                                #Package yang mau di ambil datanya
    'com.simasmobile.co.id'
]

In [ ]:
app_infos = []                                                                  #Filter package berdasatkan bahasa dan negara

for application in tqdm(app_packages):
  info = app(application, lang='id', country='id')
  del info['comments']
  app_infos.append(info)

In [ ]:
app_reviews = []                                                                #Untuk Print Datanya

for ap in tqdm(app_packages):
  for score in list(range(1,6)):
    for sort_order in [Sort.MOST_RELEVANT]:
      rvs, _ = reviews(
          ap,
          lang='id',
          country='id',
          sort=sort_order,
          count=1000,
          filter_score_with = None
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)                                      #Mendefinisikan data ke dalam DataFrameapp_reviews

In [ ]:
app_reviews_df = app_reviews_df.sort_values(by='at', ascending=False)           #Mengurutkan data berdasarkan "At" atau tanggal

In [ ]:
app_reviews_df.to_csv(path+'ScrapingData.csv', index=None, header=True)         #Set nama file datanya

In [ ]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
1716,0c5637c6-3086-475f-a656-2ec4cdce2d5c,Ronarja Zacharias,https://play-lh.googleusercontent.com/a-/AD_cM...,Terbaik👍👍,5,0,3.0238,2023-05-26 12:50:48,None,None,3.0238,most_relevant,com.simasmobile.co.id
2193,0c5637c6-3086-475f-a656-2ec4cdce2d5c,Ronarja Zacharias,https://play-lh.googleusercontent.com/a-/AD_cM...,Terbaik👍👍,5,0,3.0238,2023-05-26 12:50:48,None,None,3.0238,most_relevant,com.simasmobile.co.id
762,0c5637c6-3086-475f-a656-2ec4cdce2d5c,Ronarja Zacharias,https://play-lh.googleusercontent.com/a-/AD_cM...,Terbaik👍👍,5,0,3.0238,2023-05-26 12:50:48,None,None,3.0238,most_relevant,com.simasmobile.co.id
1239,0c5637c6-3086-475f-a656-2ec4cdce2d5c,Ronarja Zacharias,https://play-lh.googleusercontent.com/a-/AD_cM...,Terbaik👍👍,5,0,3.0238,2023-05-26 12:50:48,None,None,3.0238,most_relevant,com.simasmobile.co.id
285,0c5637c6-3086-475f-a656-2ec4cdce2d5c,Ronarja Zacharias,https://play-lh.googleusercontent.com/a-/AD_cM...,Terbaik👍👍,5,0,3.0238,2023-05-26 12:50:48,None,None,3.0238,most_relevant,com.simasmobile.co.id


In [ ]:
Scraping_Data_ASM = app_reviews_df[['content','score']]

In [ ]:
Scraping_Data_ASM.head()

,content,score
1716,Terbaik👍👍,5
2193,Terbaik👍👍,5
762,Terbaik👍👍,5
1239,Terbaik👍👍,5
285,Terbaik👍👍,5


In [ ]:
len(Scraping_Data_ASM)

2385

In [ ]:
Scraping_Data_ASM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2385 entries, 1716 to 916
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  2385 non-null   object
 1   score    2385 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 55.9+ KB


In [ ]:
Scraping_Data_ASM['score'].value_counts()

5    1195
1     790
3     170
4     120
2     110
Name: score, dtype: int64

# PRE-PROCESSING

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re

### CLEANING

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace('https\S+', ' ', case=False)          #hapus URL

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace('@\S+', ' ', case=False)              #hapus mention

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace('#\S+', ' ', case=False)              #Hapus hastag

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace("\'\w+", ' ', case=False)             #hapus next karakter

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace("[^\w\s]", ' ', case=False)           #hapus tanda baca

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace("\s(2)", ' ', case=False)             #Hapus ekstra whitespace

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [ ]:
Scraping_Data_ASM.drop_duplicates(subset ="content", keep = 'first', inplace = True)

## CASE FOLDING

In [ ]:
Scraping_Data_ASM['content'] = Scraping_Data_ASM['content'].str.lower()

## TOKENIZING

In [ ]:
from nltk.tokenize import RegexpTokenizer                                       #ini library

## NORMALIZATION

## STOPWORD REMOVAL

## STEMMING